In [1]:
import os, pathlib, shutil
from collections import Counter
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import v2
import torch.nn as nn
import numpy as np
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import train
from torchvision.transforms import InterpolationMode
from torchvision.datasets import ImageFolder

src = ""
if os.name == "nt":
    src = "D:/599DL4VProject/the_wildfire_dataset"
elif os.name == "posix":
    src = "/home/asromelo/Desktop/Projects/599_proj/the_wildfire_dataset/"

wf1TrainPath = pathlib.Path(src + '/train').resolve()
wf1ValidPath = pathlib.Path(src + '/valid').resolve()
wf1TestPath = pathlib.Path(src + '/test').resolve()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cuda = True if torch.cuda.is_available() else False

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.jit.onednn_fusion_enabled = True
torch.manual_seed(42)


# Currently using Kaggle The Wildfire Dataset and FiSmo dataset
# https://www.kaggle.com/datasets/utkarshsaxenadn/landscape-recognition-image-dataset-12k-images
# https://www.kaggle.com/datasets/uciml/forest-cover-type-dataset

c:\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorc

In [16]:
# Next W = (W + 2*pad - Kernel)/stride + 1
def downlayer(w,k,p,s):
    a1 = conv(w,k,p,s)
    a2 = conv(a1,k,p,s)
    return conv(a2,k,0,2)

def conv(w,k,p,s):
    return (w + 2*p - k)/s + 1

def convBlock(w,ksp:(int,int,int)):
    return (w + 2*ksp[2] - ksp[0])/ksp[1] + 1

def upBlock(w,ksp:(int,int,int),pad=0):
    u = 2*(w-1) - 2*pad + 1*(2-1) + 0 + 1
    return convBlock(u,ksp)

def unpooled(w,k,p,s):
    h = (w + 2*p - k)/s + 1
    return (h + 2*p - k)/s + 1

def backward(w,k,p,s):
    f = 2*(w - 1) + 2 - 2*p
    g = s*(f - 1) + k - 2*p
    return s*(g - 1) + k - 2*p

def uplayer(w,k,p,s,inner=0):
    d = 1
    innerkernel = 2
    innerstride = 2
    outpad = 0
    # Formula for Conv = (H_in - 1)*stride - 2*p + dilation*(k-1) + outpad + 1
    u = innerstride*(w - 1) - 2*inner + d*(innerkernel-1) + outpad + 1
    a1 = conv(u,k,p,s)
    return a1
    # 148 -> 72 -> 34 -> 30

def upconv(w,k,p,s):
    return s*(w-1) - 2*p + 1*(k-1) + 0 + 1

w=129
ksp = (4,1,0)

e1 = convBlock(w,ksp)
e2 = convBlock(e1,(4,2,0))
e3 = convBlock(e2,(4,2,0))
e4 = convBlock(e3,(4,2,0))
bot = convBlock(e4,ksp)
d1 = upBlock(bot,ksp)
d2 = upBlock(d1,ksp)
d3 = upBlock(d2,ksp)
d4 = upBlock(d3,ksp,1)
print("output size: ", conv(d4,1,0,1))
img_size = int(d4)
(e1,e2,e3,e4,bot,d1,d2,d3,d4)

output size:  129.0


(126.0, 62.0, 30.0, 14.0, 11.0, 19.0, 35.0, 67.0, 129.0)

In [3]:
p = 0.2
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

tfset = v2.Compose([
        v2.ToImageTensor(),
        v2.ConvertImageDtype(torch.uint8),
        v2.RandomCrop(size=512, pad_if_needed=True, padding_mode='edge'),
        v2.RandomChoice(transforms=[
            v2.Grayscale(3),
            v2.RandomRotation(45),
            v2.RandomEqualize(p),
        ], p=[0.3,0.3,0.3]),
        v2.RandomHorizontalFlip(p),
        v2.Resize(size=img_size, antialias=False),
        v2.ConvertImageDtype(dtype=torch.float),
])

testset = v2.Compose([
    v2.ToImageTensor(),
    v2.ConvertImageDtype(torch.uint8),
    v2.Resize(size=720, interpolation=InterpolationMode.BICUBIC, antialias=True),
    v2.CenterCrop(size=512),
    v2.Resize(size=img_size, antialias=False),
    v2.ConvertImageDtype(dtype=torch.float),
])

wf1Train = train.ForestFireDataset(root=str(wf1TrainPath), transform=tfset)
wf1Valid = train.ForestFireDataset(root=str(wf1ValidPath), transform=testset)
wf1Test = train.ForestFireDataset(root=str(wf1TestPath), transform=testset)

wf1Tsfire = Subset(wf1Test, [i for i, label in enumerate(wf1Test.targets) if label == 0])
wf1Tsnofire = Subset(wf1Test, [i for i, label in enumerate(wf1Test.targets) if label == 1])
wf1nofire = Subset(wf1Train, [i for i, label in enumerate(wf1Train.targets) if label == 1])

batch_size = 16
wf1TrLoader = DataLoader(dataset=wf1nofire, batch_size=batch_size, shuffle=True, num_workers=2)
wf1VaLoader = DataLoader(dataset=wf1Valid, batch_size=batch_size, num_workers=2)
wf1TsLoader = DataLoader(dataset=wf1Test, batch_size=batch_size, num_workers=2)
wf1TsFireLd = DataLoader(dataset=wf1Tsfire, batch_size=batch_size, num_workers=2)
wf1TsNoFireLd = DataLoader(dataset=wf1Tsnofire, batch_size=batch_size, num_workers=2)

In [4]:
epochs = 10
trainer = train.GANProject(img_size=img_size, debug=False, small=True, activation=nn.LeakyReLU(0.2), alternative="shufflenet_s")

In [5]:
trainer.train_model(trainLoader=wf1TrLoader, validLoader=wf1VaLoader, metric='loss', epochs=epochs, eps=0.01)

c:\Users\Narisa\Dropbox\studyDotLib\Fall2023\CPSC599\Project\train.py:128: UserWarning: Using a target size (torch.Size([16, 3, 129, 129])) that is different to the input size (torch.Size([16, 3, 114, 114])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  rec_loss = F.l1_loss(img_fake, img_real)


RuntimeError: The size of tensor a (114) must match the size of tensor b (129) at non-singleton dimension 3